# Random Quantum Circuits (RQC). Tensor Network Analysis

This notebook demonstrates the creation, sequential contraction, and parallel contraction of a RQC circuit using various algorithms such as Girvan–Newman and FlowCutter.

---



## Step 0: Loading software

 First of all, we create a new project to load all the neccesary software

In [1]:
] activate New_Project_on_QXTools;

  Activating project at `C:\Users\Usuario\OneDrive\Escriptori\anaconda_blogs\agost_2023\Novembre_desembre_2024\GitHub_resum_article_Costa_Ballena\New_Project_on_QXTools`


In [1]:
import Pkg; 
Pkg.add("QXTools")
Pkg.add("QXGraphDecompositions")
Pkg.add("QXZoo")
Pkg.add("DataStructures")
Pkg.add("QXTns")
Pkg.add("NDTensors")
Pkg.add("ITensors")
Pkg.add("LightGraphs")
Pkg.add("PyCall")




   Resolving package versions...
    Updating `C:\Users\Usuario\.julia\environments\v1.9\Project.toml`
  [84f0eee1] + QXTools v1.0.0
    Updating `C:\Users\Usuario\.julia\environments\v1.9\Manifest.toml`
  [84f0eee1] + QXTools v1.0.0
⌅ [6aa20fa7] + TensorOperations v3.2.5
        Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated -m`
Precompiling project...
  ✓ GeometryBasics
  ✓ QXTools
  ✓ NetworkLayout
  ✓ GraphRecipes
  4 dependencies successfully precompiled in 46 seconds. 263 already precompiled. 1 skipped during auto due to previous errors.
   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Usuario\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Usuario\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Usuario\.julia\environments\v1.9\Ma

In [2]:
using QXTools
using QXTns
using QXZoo
using PyCall
using QXGraphDecompositions
using LightGraphs
using DataStructures
using TimerOutputs
using ITensors
using LinearAlgebra
using NDTensors


[ Info: OMEinsum loaded the CUDA module successfully


In [3]:
# Load custom functions
include("../src/funcions_article.jl");

## Step 1: Create a RQC Circuit
We begin by creating a RQC circuit based on the user-defined number of qubits.

In [5]:
# --- Step 1: Randdom Quantum Circuit (RQC) Creation ---
       @info("How many qubits (n) do you want for the RQC circuit(nxn)?\n\n")
       
              N = readline() 
              n = parse(Int, N)
      
       @info("How deep do you want for the RQC circuit(d)?\n\n")
             N = readline()   
            d = parse(Int, N)

       
       @info("Give us a seed as a positive number, please.(seed)?\n\n")
         N = readline()  
       seed = parse(Int, N)
        # Create RQC circuit
       cct = create_rqc_circuit(n, n, d, seed, final_h=true)
       @info(" circuit RQC $(n)_$(n)_$(d) created\n\n")
     tnc = convert_to_tnc(cct)  # Convert the RQC circuit into a tensor network circuit



┌ Info: How many qubits (n) do you want for the RQC circuit(nxn)?
└ 


stdin>  5


┌ Info: How deep do you want for the RQC circuit(d)?
└ 


stdin>  16


┌ Info: Give us a seed as a positive number, please.(seed)?
└ 


stdin>  41


┌ Info:  circuit RQC 5_5_16 created
└ 


TensorNetworkCircuit(qubits => 25, gates => 382)

---

## Step 2: Sequential Contraction Using Girvan-Newman
We perform sequential contraction of the tensor network using the Girvan–Newman algorithm to get a contraction order.

In [7]:
# --- Step 2: Sequential Contraction using Girvan-Newman ---
s1 = Calcul_GN_Sequencial(cct, true)  # Perform sequential contraction
println("Sequential contraction result: ", s1)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            16.7s /  97.0%            720MiB /  97.3%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
3T. Final contraction      1    14.8s   91.1%   14.8s    676MiB   96.6%   676MiB
2T. Getting GN plan        1    1.32s    8.1%   1.32s   20.0MiB    2.9%  20.0MiB
1T. Obtaining a li...      1    126ms    0.8%   126ms   4.05MiB    0.6%  4.05MiB
────────────────────────────────────────────────────────────────────────────────
Sequential contraction result: fill(8.279270506004239e-5 + 0.00019549339310625166im)


---

## Step 3: Compare Results Using Another Contraction Plan
Here, we utilize FlowCutter to generate an alternative contraction plan and compare results.

In [8]:
# --- Step 3: Compare with Another Plan ---
using QXTools
using QXTools.Circuits

# Convert the RQC circuit into a tensor network circuit
  tnc = convert_to_tnc(cct)  
# Find a good contraction plan using FlowCutter
plan = flow_cutter_contraction_plan(tnc; time=10)

num_qubits = cct.num_qubits

# Output states
outputs = ["0" ^ num_qubits, "1" ^ num_qubits, "1" ^ (num_qubits - 2) * "01"]
eixida = outputs[1]  # Select the first output as the target

# Evaluate the probability amplitude for different outputs

@show QXTools.single_amplitude(tnc, plan, eixida)
eixida = outputs[2]  # Select the second output
@show QXTools.single_amplitude(tnc, plan, eixida)
eixida = outputs[3]  # Select the third output
@show QXTools.single_amplitude(tnc, plan, eixida)

# Perform tensor network contraction using the plan
s = contract_tn!(tnc.tn, plan)
println("Contraction result using FlowCutter plan: ", s)

# Compare results
println("Do the results match? ", s1 ≈ s)

QXTools.single_amplitude(tnc, plan, eixida) = 8.279270506004224e-5 + 0.00019549339310625185im
QXTools.single_amplitude(tnc, plan, eixida) = 0.00021936310726791653 - 0.00019493831516731146im
QXTools.single_amplitude(tnc, plan, eixida) = 4.0380865194605314e-5 + 0.00012280248186785539im
Contraction result using FlowCutter plan: fill(8.279270506004224e-5 + 0.00019549339310625185im)
Do the results match? true


---

## Step 4: Parallel Contraction Using ComPar
Finally, we use ComPar algorithms for parallel contraction.

In [9]:
# --- Step 4: Parallel Contraction using ComPar ---
# Define input and output states
num_qubits = cct.num_qubits
entrada = "0" ^ num_qubits

# Output states
outputs = ["0" ^ num_qubits, "1" ^ num_qubits, "1" ^ (num_qubits - 2) * "01"]
eixida = outputs[1]  # Select the first output as the target

n_com = 8  # Number of communities for the contraction
println("Using $n_com communities for contraction.")



Using 8 communities for contraction.


In [10]:
# Perform contraction using ComParCPU
s2 = ComParCPU(cct, entrada, eixida, n_com; timings=true, decompose=true)
println("Contraction result using ComParCPU: ", s)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            2.13s / 100.0%            130MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
2T.Parallel contra...      1    784ms   36.7%   784ms   84.9MiB   65.1%  84.9MiB
1T.Obtaining Commu...      1    739ms   34.7%   739ms   21.3MiB   16.4%  21.3MiB
3T.Final Contraction       1    610ms   28.6%   610ms   24.1MiB   18.5%  24.1MiB
────────────────────────────────────────────────────────────────────────────────
Contraction result using ComParCPU: fill(8.279270506004224e-5 + 0.00019549339310625185im)


In [11]:
# Perform contraction using ComParCPU_para
s_para = ComParCPU_para(cct, entrada, eixida, n_com; timings=true, decompose=true)
println("Contraction result using ComParCPU_para: ", s_para)

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            795ms / 100.0%           78.1MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
3T.Final contracti...      1    307ms   38.6%   307ms   5.66MiB    7.3%  5.66MiB
1T.Obtaining Commu...      1    283ms   35.6%   283ms   4.12MiB    5.3%  4.12MiB
2T.Parallel contra...      1    205ms   25.7%   205ms   68.3MiB   87.5%  68.3MiB
────────────────────────────────────────────────────────────────────────────────
Contraction result using ComParCPU_para: fill(8.279270506004232e-5 + 0.00019549339310625158im)


In [12]:
# Compare results
println("Do all the results match? ", s1 ≈ s≈ s2 ≈ s_para)

Do all the results match? true


---

### Summary
This notebook demonstrated:
1. The creation of a RQC tensor network.
2. Sequential contraction using the Girvan–Newman algorithm.
3. Alternative contraction using FlowCutter.
4. Parallel contraction with ComParCPU and ComParCPU_para.

Thank you for exploring tensor network contraction with us!